[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pun2beam/makePeriodicTableEPUB/blob/main/notebooks/colab_build.ipynb)

# Periodic Table EPUB build (Colab)
This notebook reproduces the full EPUB build pipeline described in [`spec.md`](../spec.md) using Google Colab's default Python runtime.
**Usage notes**:
- Open this notebook in Google Colab (File → Open notebook → GitHub).
- (Optional) Connect your Google Drive if you want to persist the generated EPUB/cover files.
- Run the cells from top to bottom; the pipeline will download the latest Wikipedia data, normalize it, fetch per-element summaries, build the SVG cover, rasterize it, and assemble the EPUB package inside `book/dist/`.


In [ ]:
#@title Install Python dependencies
%cd /content/
!git clone https://github.com/pun2beam/makePeriodicTableEPUB.git
%cd /content/makePeriodicTableEPUB
!pip install --quiet -r requirements.txt

In [ ]:
#@title Install system fonts for Japanese text
!apt-get update -y
!apt-get install -y fonts-noto-cjk
!fc-cache -fv


In [ ]:
%cd /content/makePeriodicTableEPUB
!git pull

## Optional: Mount Google Drive
Uncomment the cell below if you want the generated assets to be saved to Google Drive instead of the Colab ephemeral filesystem.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/path/to/your/workdir

## Build the EPUB
The commands below follow the workflow defined in Section 4 of the specification. They can be rerun to refresh the data or rebuild the package after modifying templates or scripts.

In [ ]:
#@title Run data fetch, normalization, and build scripts
# English Version
!python scripts/fetch_wiki.py --lang en --page "List of chemical elements" --output-file list-of-chemical-elements-en-rest.json
!python scripts/normalize.py --input data/raw/list-of-chemical-elements-en-rest.json
!python scripts/fetch_wiki.py --lang en --page "List of chemical elements" --elements-from data/tables.json --elements-json data/elements.en.json
!python scripts/build_cover_svg.py --in data/tables.json --out assets/gen/cover_en.svg
!python scripts/license_attribution.py --out book/OEBPS/attribution.xhtml
!python scripts/rasterize_cover.py --in assets/gen/cover_en.svg --out book/dist/cover_en_2560x1600.jpg --width 1600 --height 2560
!python scripts/build_epub.py --cover book/dist/cover_en_2560x1600.jpg --out book/dist/PeriodicTable.en.epub --element-data data/elements.en.json

# Japanese Version
!python scripts/fetch_wiki.py --lang ja --page "元素の一覧" --output-file list-of-chemical-elements-ja-rest.json
!python scripts/normalize.py --input data/raw/list-of-chemical-elements-ja-rest.json
!python scripts/fetch_wiki.py --lang ja --page "元素の一覧" --elements-from data/tables.json --elements-json data/elements.ja.json
!python scripts/build_cover_svg.py --in data/tables.json --out assets/gen/cover_ja.svg
!python scripts/license_attribution.py --out book/OEBPS/attribution.xhtml
!python scripts/rasterize_cover.py --in assets/gen/cover_ja.svg --out book/dist/cover_ja_2560x1600.jpg --width 1600 --height 2560
!python scripts/build_epub.py --cover book/dist/cover_ja_2560x1600.jpg --out book/dist/PeriodicTable.ja.epub --element-data data/elements.ja.json


## Inspect generated files
Run the following cell to list the distribution folder and confirm that the EPUB and cover JPEG were produced successfully.

In [ ]:
#@title Check build outputs
!ls -lh book/dist

In [ ]:
# !rm -rf /content/makePeriodicTableEPUB/data
# !rm -rf /content/makePeriodicTableEPUB/book/
# !rm -rf /content/makePeriodicTableEPUB/makePeriodicTableEPUB/
